# 01. 데이터 로드

In [75]:
# | code-fold : true

import pandas as pd
import os
import numpy as np
import warnings

warnings.filterwarnings(action = "ignore")

In [76]:
file_list = os.listdir("total_data")

In [77]:
for i in range(len(file_list)) : 
    exec(f'df{i+1} = pd.read_csv("total_data/" + file_list[{i}],encoding = "euc-kr")')

In [78]:
df3.rename(columns = {"학교코드_공시연도" : "key"}, inplace = True)

# 02. df2 데이터 전처리

## (1) key값 수정

In [79]:
#df2.info()

`-` df2 데이터를 살펴본 결과 3번째 부터 32번째 컬럼은 필요 없어보임

In [80]:
drop_col = df2.columns[3:33].tolist()

df2.drop(drop_col, axis = 1, inplace = True)
df2.dropna(subset = ["key"], inplace = True)

In [81]:
df2.shape

(154031, 31)

`1` 전처리 과정에서 key 값 결합이 잘못된 것 같음.. 다시 분할 한 후 합쳐 주자

In [82]:
df2["school_code"] = [i.split("-")[0] for i in df2["key"]]
df2["year"] = [i.split("-")[1] for i in df2["key"]]
df2.year.replace("nan",0, inplace =True)

In [83]:
df2.year = df2.year.astype(float).astype(int)

In [84]:
df2 = df2.loc[df2.year !=0 ,:]

`2` 합치기

In [85]:
df2["key"] = [i + "-" + str(j) for i,j in zip(df2["school_code"], df2["year"])]

In [86]:
df2.key.unique()

array(['S000003511-2021', 'S000003563-2021', 'S010000738-2021', ...,
       'S010006027-2024', 'S010006380-2024', 'S010005864-2024'],
      dtype=object)

In [87]:
df2.drop_duplicates(subset = ["key"],inplace = True)

`3` 필요없는 변수 삭제

In [88]:
del_col = ["장학금인원","장학금금액","학비지원인원","학비지원금액","인원합계","금액합계_x","금액합계_y", "school_code", "year",
            "건수합계", "지출금액합계", "금전및 유가증권건수", "도서및 물품건수", "수목시설및 재산건수","교육청비율", "자치단체비율",
            "금액계", "비율계", "분류"]

df2.drop(del_col, axis =1, inplace = True)

In [89]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5475 entries, 0 to 151459
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   key             5475 non-null   object 
 1   정보공시 학교코드       5475 non-null   object 
 2   학교명             5475 non-null   object 
 3   교육청금액           5365 non-null   float64
 4   자치단체금액          5365 non-null   float64
 5   보호자금액           5365 non-null   float64
 6   보호자비율           5365 non-null   float64
 7   금전및 유가증권금액      3650 non-null   float64
 8   도서및 물품금액        3650 non-null   float64
 9   수목시설및 재산금액      3650 non-null   float64
 10  학교교육시설 보수확충금액   3650 non-null   float64
 11  교육용기자재 및도서금액    3650 non-null   float64
 12  학교체육학예 활동금액     3650 non-null   float64
 13  학생복지및 자치활동지원금액  3650 non-null   float64
dtypes: float64(11), object(3)
memory usage: 641.6+ KB


`-` 참고 : 교육청금액과 자치단체금액, 보호자 금액은 `급식비 집행 실적 현황`에 대한 내용임

## (2) 결측치 채우기

결측치는 금액과 관련된 내용이니 전부 `0`으로 채우자...(실제로 예산을 집행안한 걸 수 도 있으니까...)

In [90]:
df2.fillna(0, inplace = True)

In [91]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5475 entries, 0 to 151459
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   key             5475 non-null   object 
 1   정보공시 학교코드       5475 non-null   object 
 2   학교명             5475 non-null   object 
 3   교육청금액           5475 non-null   float64
 4   자치단체금액          5475 non-null   float64
 5   보호자금액           5475 non-null   float64
 6   보호자비율           5475 non-null   float64
 7   금전및 유가증권금액      5475 non-null   float64
 8   도서및 물품금액        5475 non-null   float64
 9   수목시설및 재산금액      5475 non-null   float64
 10  학교교육시설 보수확충금액   5475 non-null   float64
 11  교육용기자재 및도서금액    5475 non-null   float64
 12  학교체육학예 활동금액     5475 non-null   float64
 13  학생복지및 자치활동지원금액  5475 non-null   float64
dtypes: float64(11), object(3)
memory usage: 641.6+ KB


****

# 03. 합치기

In [92]:
del_col = ["정보공시 학교코드", "학교명"]

In [93]:
df2.drop(del_col, axis = 1, inplace = True)
df3.drop(del_col, axis = 1, inplace = True)

In [94]:
df3.drop("지역", axis = 1, inplace = True)

In [95]:
temp = pd.merge(df1,df2, on = "key", how = "left")
temp = pd.merge(temp,df3, on = "key", how = "left")
temp = pd.merge(temp,df4, on = "key", how = "left")
temp.fillna(0, inplace = True)

# 04. 합친 데이터 살펴보기

In [96]:
temp.shape

(5475, 77)

In [97]:
#temp.info()

## (1) 필요없는 컬럼 삭제 1

`-` 학교별 직원 현황은 별로 의미가 없어 

* 수업 교원 1인당 학생 수가 있으므로!

In [98]:
d_col = ["일반직계", "별정직계(2018년이후)", "기타직계(2019년이후)"]

temp.drop(d_col, axis = 1, inplace = True)

## (2) 필요없는 컬럼 삭제 2

In [99]:
#temp.info()

`-` 특수, 순회학급, 교사 구분 변수 제외

In [100]:
#print(temp.columns.tolist())

In [101]:
d_col = ['특수학급 학급당 학생수', '순회학급 학급당 학생수', '수석교사(계)', '전문상담교사(계)', '사서교사(계)', '보건교사(계)', '영양교사(계)']
temp.drop(d_col, axis = 1, inplace = True)

## (3) 필요없는 컬럼 삭제 3

In [102]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5475 entries, 0 to 5474
Data columns (total 67 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   key                                        5475 non-null   object 
 1   지역                                         5475 non-null   object 
 2   정보공시 학교코드                                  5475 non-null   object 
 3   학교명                                        5475 non-null   object 
 4   교과교실제(지정?운영)                               5475 non-null   int64  
 5   자율학교(지정?운영)                                5475 non-null   int64  
 6   수준별 수업(운영)                                 5475 non-null   int64  
 7   전체학생수                                      5475 non-null   float64
 8   연간1인당 보건실이용건수                              5475 non-null   float64
 9   수업교원 1인당 학생수                               5475 non-null   float64
 10  체육장 개방여부                

`-` 일단 여기까지...

In [103]:
#pd.DataFrame({"변수" : temp.columns.tolist()}).to_csv("변수명.csv", index = False, encoding = "euc-kr")

In [104]:
#temp.to_csv("통합1.csv", index = False, encoding = "euc-kr")

`-` 연간1인당 보건실이용건수는 학교 평가시 별 의미가 없는 변수인 것 같음.....

In [105]:
print(temp.columns.tolist())

['key', '지역', '정보공시 학교코드', '학교명', '교과교실제(지정?운영)', '자율학교(지정?운영)', '수준별 수업(운영)', '전체학생수', '연간1인당 보건실이용건수', '수업교원 1인당 학생수', '체육장 개방여부', '체육관 개방여부', '강당 개방여부', '일반교과교실 개방여부', '특별교실 개방여부', '시청각실 개방여부', '체력 1등급 (%)', '체력 2등급 (%)', '체력 3등급 (%)', '체력 4등급 (%)', '체력 5등급 (%)', '교육청금액', '자치단체금액', '보호자금액', '보호자비율', '금전및 유가증권금액', '도서및 물품금액', '수목시설및 재산금액', '학교교육시설 보수확충금액', '교육용기자재 및도서금액', '학교체육학예 활동금액', '학생복지및 자치활동지원금액', '지도교사수-동아리?학생자치활동', '지도교사수-또래활동', '지도교사수-교육주간 활동', '지도교사수-기타 학교폭력 예방활동', '참여학생수-동아리?학생자치활동', '참여학생수-또래활동', '참여학생수-교육주간 활동', '참여학생수-기타 학교폭력 예방활동', '전체학생수_안전교육 계획 및 실시현황', '생활안전교육 시간(단위활동차시)-합계', '교통안전교육 시간(단위활동차시)-합계', '폭력예방및신변보호교육 시간(단위활동차시)-합계', '약물중독예방교육 시간(단위활동차시) - 합계 (2020년 이후)', '지능정보서비스과의존관련교육 시간(단위활동차시) - 합계 (2023년 이후)', '직업안전교육 시간(단위활동차시)-합계', '응급처치교육 시간(단위활동차시)-합계', '주 출입구 접근로 설치여부', '장애인 전용 주차구역 지정여부', '주 출입구 높이차이 제거여부', '출입구(문) 설치유무', '복도', '계단/승강기/경사로/휠체어리프트 유무', '장애인용 대변기 설치여부', '장애인용 소변기 설치여부', '점자블록 설치여부', '유도 및 안내설비 설치여부', '경보 및 피난설비 설치여부', '상담실적(내부상담전문가)', '상담실적(외부상

In [106]:
del_col = ['체력 1등급 (%)', '체력 2등급 (%)', '체력 3등급 (%)', '체력 4등급 (%)', '체력 5등급 (%)',
'교육청금액', '자치단체금액', '보호자금액', '금전및 유가증권금액', '도서및 물품금액', '수목시설및 재산금액', '학교교육시설 보수확충금액', 
 '교육용기자재 및도서금액', '학교체육학예 활동금액', '학생복지및 자치활동지원금액']

In [107]:
temp.drop(del_col, axis = 1, inplace = True)

In [108]:
temp.to_csv("통합2.csv", encoding = "euc-kr", index = False)

In [110]:
temp.shape

(5475, 52)

`-` 어떤 카테고리가 들어갔으면 좋겠다. 어떤 시각화가 들어갔으면 좋은지...